In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
import meshcat

# Setup meshcat

In [4]:
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
from meshcat import Visualizer
from pydrake.all import ConnectMeshcatVisualizer

proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

do_viz = False

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/


# Build plant

In [5]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
                                      delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6010...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-4,
    'iris_plane_pullback': 1e-6,
    'iris_max_faces': -1
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1
                        /2)
regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


geomA=iiwa::link7, geomB=shelves::top
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.14471316 -0.66482345  0.73909617], growth = 147061223.6237918
snopt_example=[ 0.14526163 -0.66347859  0.74079434], growth = 147061480.4126292
snopt_example=[ 0.14664268 -0.66010327  0.74507106], growth = 147064396.912189
snopt_example=[ 0.1441652  -0.66616957  0.73739972], growth = 147061480.1075334
snopt_example=[ 0.1427902  -0.6695586   0.73314332], growth = 147064386.88576454
snopt_example=[ 0.14284248 -0.66942796  0.73330732], growth = 147064149.01532334
snopt_example=[ 0.14289679 -0.66929207  0.7334779 ], growth = 147063911.5266152
snopt_example=[ 0.14295514 -0.66915071  0.7336557 ], growth = 147063674.48673064
snopt_example=[ 0.14301522 -0.66900236  0.73384206], growth = 147063437.98193246
snopt_example=[ 0.14307842 -0.66884624  0.73403819], growth = 147063202.12595516
sno

geomA=wsg::body_collision, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
geomA=wsg::right_collision, geomB=shelves::shelf_lower
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::shelf_upper
0
geomA=iiwa::link7, geomB=shelves::top
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::body_collision, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::

snopt_example=[-0.06967966 -1.24912377 -0.13461898], growth = 0.5907411156217757
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
geomA=wsg::right_collision, geomB=shelves::shelf_lower
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::shelf_upper
terminating because a required containment point would have not been contained
Time:   2.43 	Volume:   3.30 	Center: [-7.89714714e-01 -1.23542089e+00  7.44200257e-11]
geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.14340093 -0.84451312 -0.11291406], growth = 18466675.42165816
snopt_example=[ 0.1434027  -0.844515   -0.11292629], growth = 18466588.036848873
snopt_example=[ 0.14340064 -0.84451527 -0.11291633], growth = 18466495.495584697
snopt_example=[ 0.14340047 -0.84451629 -0.11291726], growth = 18466409.

In [9]:
# plot regions
if do_viz:
    viz_utils.plot_regions(vis2, regions, ellipses)

    # plot seedpoints
    for i in range(seed_points.shape[0]):
        vis2['iris']['seedpoints']["seedpoint"+str(i)].set_object(
                    meshcat.geometry.Sphere(0.05), meshcat.geometry.MeshLambertMaterial(color=0x0FB900))
        vis2['iris']['seedpoints']["seedpoint"+str(i)].set_transform(
                    meshcat.transformations.translation_matrix(seed_points[i,:]))
    

## Marching cubes in t space

In [10]:
#evaluate constraints
def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q

def convert_q_to_t(q):
    return np.tan(np.divide(q,2))

ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)
def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def eval_cons_rational(t0, t1, t2, c, tol):
    q = convert_t_to_q(np.array([t0, t1, t2]).reshape(1,-1)).squeeze() 
    return col_func_handle(*q)
   
col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)


In [11]:
#marching cubes
if do_viz:
    N = 50
    vertices, triangles = mcubes.marching_cubes_func(tuple(iris_generator.t_lower_limits), 
                                                     tuple(iris_generator.t_upper_limits),
                                                     N, N, N, col_func_handle_rational, 0.5)

    vis2["collision_constraint"].set_object(
                meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
                meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))

In [12]:
from pydrake.all import MathematicalProgram
(geomA, geomB)  = next(iter(iris_generator.pairs))

region_to_prog_map, collision_pair_to_plane_map,\
collision_pair_to_positive_constraint_map, pos_poly_to_lagrange_mult_map\
=iris_generator._initialize_certifier(1)
# initializing the maps actually doesn't seem to save that much time. maybe it
# has a memory impact but for the inconvenience it gives its unclear whether this is useful


In [ ]:
region_to_prog_map = iris_generator._build_certification_problems_from_maps(region_to_prog_map, 
                                                                            pos_poly_to_lagrange_mult_map,
                                               strict_pos_tol = 1e-5)


In [ ]:
region = regions[3]

In [ ]:
t0 = time.time()
result = iris_generator.mosek.Solve(region_to_prog_map[region])
t1 = time.time()
print(f"Solved in {t1-t0}seconds")

Notes:
Currently have build certifiaction problems from maps done. Now can solve. Current design strategy means we should not solve the programs in parallel since all the programs point
to the same variables. This saves a lot of memory and (hopefully) speeds up the construction substantially